# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 12:53PM,238883,18,CLRSPC-30598325,"ClearSpec, LLC",Released
1,Jun 6 2022 12:36PM,238881,12,HH6197,Hush Hush,Released
2,Jun 6 2022 12:23PM,238880,19,SD06072022,"SD Head USA, LLC",Released
3,Jun 6 2022 12:20PM,238879,20,8086571,"ACI Healthcare USA, Inc.",Released
4,Jun 6 2022 12:20PM,238879,20,8070515-BO,"ACI Healthcare USA, Inc.",Released
5,Jun 6 2022 12:20PM,238879,20,8086570-BO,"ACI Healthcare USA, Inc.",Released
6,Jun 6 2022 12:20PM,238879,20,8086613-BO,"ACI Healthcare USA, Inc.",Released
7,Jun 6 2022 12:20PM,238879,20,8091859-BO,"ACI Healthcare USA, Inc.",Released
8,Jun 6 2022 12:20PM,238879,20,8091865-BO,"ACI Healthcare USA, Inc.",Released
9,Jun 6 2022 12:20PM,238879,20,8091870-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,238878,Released,1
25,238879,Released,44
26,238880,Released,1
27,238881,Released,1
28,238883,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238878,NaN,NaN,1.0
238879,NaN,NaN,44.0
238880,NaN,NaN,1.0
238881,NaN,NaN,1.0
238883,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5.0,2.0,6.0
238822,7.0,6.0,1.0
238832,0.0,0.0,39.0
238836,0.0,0.0,51.0
238837,0.0,0.0,44.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5,2,6
238822,7,6,1
238832,0,0,39
238836,0,0,51
238837,0,0,44


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,2,6
1,238822,7,6,1
2,238832,0,0,39
3,238836,0,0,51
4,238837,0,0,44


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,2,6
1,238822,7,6,1
2,238832,,,39
3,238836,,,51
4,238837,,,44


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 12:53PM,238883,18,"ClearSpec, LLC"
1,Jun 6 2022 12:36PM,238881,12,Hush Hush
2,Jun 6 2022 12:23PM,238880,19,"SD Head USA, LLC"
3,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc."
47,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.
48,Jun 6 2022 12:05PM,238875,10,Eywa Pharma Inc.
68,Jun 6 2022 11:51AM,238874,20,"ACI Healthcare USA, Inc."
134,Jun 6 2022 11:42AM,238873,17,Alexza Pharmaceuticals
135,Jun 6 2022 11:34AM,238872,19,"AdvaGen Pharma, Ltd"
146,Jun 6 2022 11:21AM,238869,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 6 2022 12:53PM,238883,18,"ClearSpec, LLC",,,1
1,Jun 6 2022 12:36PM,238881,12,Hush Hush,,,1
2,Jun 6 2022 12:23PM,238880,19,"SD Head USA, LLC",,,1
3,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",,,44
4,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,,,1
5,Jun 6 2022 12:05PM,238875,10,Eywa Pharma Inc.,,2,18
6,Jun 6 2022 11:51AM,238874,20,"ACI Healthcare USA, Inc.",,,66
7,Jun 6 2022 11:42AM,238873,17,Alexza Pharmaceuticals,,,1
8,Jun 6 2022 11:34AM,238872,19,"AdvaGen Pharma, Ltd",,,11
9,Jun 6 2022 11:21AM,238869,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:53PM,238883,18,"ClearSpec, LLC",1,,
1,Jun 6 2022 12:36PM,238881,12,Hush Hush,1,,
2,Jun 6 2022 12:23PM,238880,19,"SD Head USA, LLC",1,,
3,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",44,,
4,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,1,,
5,Jun 6 2022 12:05PM,238875,10,Eywa Pharma Inc.,18,2,
6,Jun 6 2022 11:51AM,238874,20,"ACI Healthcare USA, Inc.",66,,
7,Jun 6 2022 11:42AM,238873,17,Alexza Pharmaceuticals,1,,
8,Jun 6 2022 11:34AM,238872,19,"AdvaGen Pharma, Ltd",11,,
9,Jun 6 2022 11:21AM,238869,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:53PM,238883,18,"ClearSpec, LLC",1,,
1,Jun 6 2022 12:36PM,238881,12,Hush Hush,1,,
2,Jun 6 2022 12:23PM,238880,19,"SD Head USA, LLC",1,,
3,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",44,,
4,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:53PM,238883,18,"ClearSpec, LLC",1.0,NaN,NaN
1,Jun 6 2022 12:36PM,238881,12,Hush Hush,1.0,NaN,NaN
2,Jun 6 2022 12:23PM,238880,19,"SD Head USA, LLC",1.0,NaN,NaN
3,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",44.0,NaN,NaN
4,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:53PM,238883,18,"ClearSpec, LLC",1.0,0.0,0.0
1,Jun 6 2022 12:36PM,238881,12,Hush Hush,1.0,0.0,0.0
2,Jun 6 2022 12:23PM,238880,19,"SD Head USA, LLC",1.0,0.0,0.0
3,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",44.0,0.0,0.0
4,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2388491,248.0,8.0,7.0
12,238881,1.0,0.0,0.0
15,238852,2.0,0.0,0.0
17,238873,1.0,0.0,0.0
18,238883,1.0,0.0,0.0
19,1194296,19.0,3.0,5.0
20,955448,111.0,1.0,0.0
22,238869,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2388491,248.0,8.0,7.0
1,12,238881,1.0,0.0,0.0
2,15,238852,2.0,0.0,0.0
3,17,238873,1.0,0.0,0.0
4,18,238883,1.0,0.0,0.0
5,19,1194296,19.0,3.0,5.0
6,20,955448,111.0,1.0,0.0
7,22,238869,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,248.0,8.0,7.0
1,12,1.0,0.0,0.0
2,15,2.0,0.0,0.0
3,17,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,19.0,3.0,5.0
6,20,111.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,248.0
1,12,Released,1.0
2,15,Released,2.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,17,18,19,20,22
Status,,,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
Executing,8.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
Released,248.0,1.0,2.0,1.0,1.0,19.0,111.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,17,18,19,20,22
0,Completed,7.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,8.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
2,Released,248.0,1.0,2.0,1.0,1.0,19.0,111.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,17,18,19,20,22
0,Completed,7.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,8.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
2,Released,248.0,1.0,2.0,1.0,1.0,19.0,111.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()